In [47]:
import numpy as np 
import pandas as pd
import os
import scipy.stats as ss
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway
import pandas as pd
import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [48]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [55]:
# 데이터 불러오기
past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')
store = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/store.csv')
# 데이터 제거
store = store.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1)

# 데이터 병합
data = pd.merge(left = past_sales,right = store, on = 'Store', how = 'right')

# 날짜
data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
data['Month']=pd.DatetimeIndex(data.Date).month
data['Day']=pd.DatetimeIndex(data.Date).day
data = data.drop(['Date'],axis=1)

# 데이터 가공 : StateHoliday
data['StateHoliday'] = data['StateHoliday'].replace({0:'d'}) # 0 값은 d로 대체
data['StateHoliday'] = data['StateHoliday'].replace({'0':'d'})# 0 값은 d로 대체


# 결측값 처리
data = data.dropna(axis=0)

# store 이름 제거
data = data.drop(['Store'],axis=1)

# 데이터 분리(수치형, 범주형)
digital_cols = ['Sales', 'Customers', 'CompetitionDistance', 'Month', 'Day']
digital_data = data[digital_cols]
digital_data = pd.DataFrame(digital_data.values, columns = digital_cols)

# 표준화
scaler = StandardScaler().fit(digital_data)
digital_data = pd.DataFrame(scaler.transform(digital_data),index = digital_data.index, columns = digital_cols)

categorey_clos = data.columns.drop(digital_cols)
categorey_data = data[categorey_clos]
categorey_data = pd.DataFrame(categorey_data.values, columns = categorey_clos)

# 인코딩
onehotencoding_cols = ['StateHoliday', 'StoreType', 'Assortment', 'DayOfWeek']
categorey_data = pd.get_dummies(data = categorey_data, columns = onehotencoding_cols)

# 데이터 병합
data = pd.concat([digital_data, categorey_data], axis = 1)

# 형 통일
data = data.astype({'Promo2':'int', 'StateHoliday_b':'int', 'StateHoliday_c':'int', 'StateHoliday_d':'int', 'StoreType_a':'int', 'StoreType_b':'int', 'StoreType_c':'int', 'StoreType_d':'int'})
data = data.astype({'Assortment_a':'int', 'Assortment_b':'int', 'Assortment_c':'int', 'DayOfWeek_1':'int', 'DayOfWeek_2':'int', 'DayOfWeek_3':'int', 'DayOfWeek_4':'int', 'DayOfWeek_5':'int', 'DayOfWeek_6':'int', 'DayOfWeek_7':'int'})

pd.options.display.max_columns = None
data


C:\Users\rlaeo\AppData\Local\Temp\ipykernel_22384\1650671467.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')


,Sales,Customers,CompetitionDistance,Month,Day,Open,Promo,SchoolHoliday,Promo2,StateHoliday_a,StateHoliday_b,StateHoliday_c,StateHoliday_d,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,-0.133481,-0.169168,-0.539198,0.346614,1.740763,1,1,1,0,False,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1,-0.196581,-0.188537,-0.539198,0.346614,1.626967,1,1,1,0,False,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0
2,-0.258383,-0.238037,-0.539198,0.346614,1.513171,1,1,1,0,False,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0
3,-0.198918,-0.158407,-0.539198,0.346614,1.399374,1,1,1,0,False,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
4,0.084382,-0.046494,-0.539198,0.346614,1.285578,1,1,1,0,False,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014562,-0.261239,-0.634036,-0.010380,-1.457264,-1.217938,1,0,1,1,False,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1014563,-0.321223,-0.662014,-0.010380,-1.457264,-1.331734,1,0,1,1,False,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0
1014564,-0.384323,-0.717970,-0.010380,-1.457264,-1.445531,1,0,1,1,False,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0
1014565,-0.540124,-0.707209,-0.010380,-1.457264,-1.559327,1,0,1,1,False,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0


In [50]:
target_col = 'Sales'
features_cols = data.columns.drop(target_col)        
features = data[features_cols]
target = data[target_col]
target = pd.DataFrame(target.values.reshape(-1,1))
features = features.values
target = target.values

In [51]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.4, shuffle=True)
x_test,x_valid,y_test,y_valid = train_test_split(x_test, y_test, test_size=0.5, shuffle=False)

In [52]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_valid = x_valid.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
y_valid = y_valid.astype(np.float32)
print('train_data의 row : %s, column : %s'%(x_train.shape[0],x_train.shape[1]))
print('test_data의 row : %s, column : %s'%(x_test.shape[0],x_test.shape[1]))
print('val_data의 row : %s, column : %s'%(x_valid.shape[0],x_valid.shape[1]))
data

train_data의 row : 610325, column : 40
test_data의 row : 203442, column : 40
val_data의 row : 203442, column : 40


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,StateHoliday_0,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,Month,Day,PromoInterval_Jan,PromoInterval_Feb,PromoInterval_Mar,PromoInterval_Apr,PromoInterval_May,PromoInterval_Jun,PromoInterval_Jul,PromoInterval_Aug,PromoInterval_Sep,PromoInterval_Oct,PromoInterval_Nov,PromoInterval_Dec
0,1,5,5263,555,1,1,1,0.016482,0.727273,0.93913,0,NaN,NaN,False,True,False,False,False,False,False,True,False,True,False,False,7,31,0,0,0,0,0,0,0,0,0,0,0,0
1,1,4,5020,546,1,1,1,0.016482,0.727273,0.93913,0,NaN,NaN,False,True,False,False,False,False,False,True,False,True,False,False,7,30,0,0,0,0,0,0,0,0,0,0,0,0
2,1,3,4782,523,1,1,1,0.016482,0.727273,0.93913,0,NaN,NaN,False,True,False,False,False,False,False,True,False,True,False,False,7,29,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2,5011,560,1,1,1,0.016482,0.727273,0.93913,0,NaN,NaN,False,True,False,False,False,False,False,True,False,True,False,False,7,28,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,6102,612,1,1,1,0.016482,0.727273,0.93913,0,NaN,NaN,False,True,False,False,False,False,False,True,False,True,False,False,7,27,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,4771,339,1,0,1,0.070280,NaN,NaN,1,0.428571,0.5,False,True,False,False,False,False,False,False,True,False,False,True,1,5,0,0,1,0,0,1,0,0,1,0,0,1
1017205,1115,5,4540,326,1,0,1,0.070280,NaN,NaN,1,0.428571,0.5,False,True,False,False,False,False,False,False,True,False,False,True,1,4,0,0,1,0,0,1,0,0,1,0,0,1
1017206,1115,4,4297,300,1,0,1,0.070280,NaN,NaN,1,0.428571,0.5,False,True,False,False,False,False,False,False,True,False,False,True,1,3,0,0,1,0,0,1,0,0,1,0,0,1
1017207,1115,3,3697,305,1,0,1,0.070280,NaN,NaN,1,0.428571,0.5,False,True,False,False,False,False,False,False,True,False,False,True,1,2,0,0,1,0,0,1,0,0,1,0,0,1


In [53]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_dim = x_train.shape[1],activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
             loss='mse',
             metrics=['mean_absolute_error'])

model_path = 'C:/project/rossmann-store-sales'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_path , monitor='val_mean_absolute_error', verbose=1, save_best_only=True)
# early_stopping = EarlyStopping(monitor='val_loss', patience=6)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 256)               10496     
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 batch_normalization_8 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 256)               65792     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                      

In [54]:
early_stopping = EarlyStopping()

start = datetime.datetime.now()

history = model.fit(x_train, y_train,
                    validation_data=(x_valid, y_valid),
                    batch_size=64,
                    epochs=25,
)

end = datetime.datetime.now()
time = end-start
print("처리시간 : ", time)

Epoch 1/25
1160/9537 [==>...........................] - ETA: 38s - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: 